In [2]:
import pandas as pd
import geopy.distance

In [143]:
# df = pd.read_csv("201809-citibike-tripdata.csv")
df = pd.read_csv("201809-citibike-tripdata.csv",  parse_dates=[1,2], dtype={"start station id": "Int64", "end station id": "Int64"})
df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.269,2018-09-01 00:27:20.634,252,MacDougal St & Washington Sq,40.732264,-73.998522,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.281,2018-09-01 00:02:23.481,314,Cadman Plaza West & Montague St,40.693830,-73.990539,3242,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.649,2018-09-01 00:55:58.547,3142,1 Ave & E 62 St,40.761227,-73.960940,3384,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.746,2018-09-01 00:07:38.583,308,St James Pl & Oliver St,40.713079,-73.998512,3690,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.315,2018-09-01 02:21:25.308,345,W 13 St & 6 Ave,40.736494,-73.997044,380,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


### 13.5.3 Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [133]:
df.shape

(1877884, 13)

### 13.5.4 Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [134]:
round(df["tripduration"].mean()/60,2)

16.13

### 13.5.5 Сколько поездок начались и закончились в той же самой станции?

In [135]:
df[df["start station name"]==df["end station name"]].shape[0]

41364

### 13.5.6 Какой самый используемый байк(bikeid) в городе по количеству поездок?

In [136]:
df.groupby(["bikeid"], as_index=False).count().sort_values("tripduration", ascending=False)["bikeid"].head(n=1)

10105    33875
Name: bikeid, dtype: int64

### 13.5.7 Найдите идентификатор велосипеда (bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [137]:
df.groupby(["bikeid"], as_index=False).mean().sort_values("tripduration", ascending=False).head(n=1)["bikeid"]

1840    17548
Name: bikeid, dtype: int64

### 13.5.8 Сколько строк, в которых отсутствуют данные о start station id?

In [138]:
df[df["start station id"].isnull()]["bikeid"].count()

716

### 13.5.9 Какова средняя продолжительность поездки в минутах в зависимости от типа подписки c точностью до 2 знака?

In [139]:
result = df.groupby(["usertype"], as_index=False).mean()[["usertype", "tripduration"]]
result["tripduration"] = round(result["tripduration"]/60, 2)
result

,usertype,tripduration
0,Customer,33.42
1,Subscriber,13.33


### 13.5.10 Найдите среднюю длину поездок в километрах с точностью до 2 знака, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id).

Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)

In [140]:
trips = df[df["start station id"] != df["end station id"]]

In [141]:
def get_distance(row):
    start_point = (row["start station latitude"], row["start station longitude"])
    finish_point = (row["end station latitude"], row["end station longitude"])
    return geopy.distance.distance(start_point, finish_point).km

round(trips.apply(get_distance, axis=1).mean(),2) #very slow solution

1.85

### 13.5.11 Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера включительно

In [144]:
df.set_index(keys=['starttime'],inplace=True)
df.between_time("18:00", "20:00").groupby("start station id",as_index=False).count().sort_values("tripduration",ascending=False)["start station id"].head(1)

250    519
Name: start station id, dtype: int64

### 13.5.12 Выберите идентификаторы станций(end station id), в которые приезжают с 6 до 10 утра включительно

In [145]:
df.set_index(keys='stoptime',inplace=True)
target_stations = {3116,3106,3140,369}
found_stations = df.between_time("6:00", "10:00")["end station id"].unique()
target_stations.intersection(set(found_stations))

{369, 3106, 3116, 3140}